In [ ]:
import json
import glm
import math

In [ ]:
robot_path = "/home/samuel/CLionProjects/EvoMotion/evo_motion_model/resources/skeleton/spider_new.json"
output_path = "new_json_robot.json"

In [ ]:
with open(robot_path, "r") as f:
    robot_data = json.load(f)

In [ ]:
def to_mat4(json_transformation) -> glm.mat4:
    pos = glm.vec3(json_transformation["translation"]["x"], json_transformation["translation"]["y"], json_transformation["translation"]["z"])

    rotation = json_transformation["rotation"]
    rot_point = glm.vec3(rotation["point"]["x"], rotation["point"]["y"], rotation["point"]["z"])
    rot_axis = glm.vec3(rotation["axis"]["x"], rotation["axis"]["y"], rotation["axis"]["z"])
    angle_radian = math.pi * rotation["angle_degree"] / 180.0

    translation_to_origin = glm.translate(-rot_point)
    rotation_matrix = glm.rotate(angle_radian, rot_axis)
    translation_back = glm.translate(rot_point)
    translation_to_position = glm.translate(pos)

    return translation_to_position * translation_back * rotation_matrix * translation_to_origin

def decompose(mat4: glm.make_mat4) -> tuple[glm.vec3, glm.quat, glm.vec3]:
    new_translate = glm.vec3(0.0)
    new_rotate = glm.quat()
    new_scale = glm.vec3(1.0)
    if glm.decompose(mat4, new_scale, new_rotate, new_translate, glm.vec3(1.0), glm.vec4(1.0)):
        return new_translate, new_rotate, new_scale
    raise Exception("Decomposition failed")

In [ ]:
queue = [(robot_data["skeleton"], glm.mat4(1.0), [], None)]

new_members = []
new_constraints = []

while len(queue) > 0:
    (curr_member, parent_model_mat, parent_names, curr_constraint) = queue.pop(0)

    # Member
    curr_model_mat = parent_model_mat * to_mat4(curr_member["transformation"])

    curr_translate, curr_rotate, _ = decompose(curr_model_mat)

    new_name = parent_names + [curr_member["name"]]

    friction = 0.5
    ignore_col = False
    if "option" in curr_member:
        if "friction" in curr_member["option"]:
            friction = curr_member["option"]["friction"]
        if "ignore_collision" in curr_member["option"]:
            ignore_col = curr_member["option"]["ignore_collision"]

    new_member = {
        "name": "_".join(new_name),
        "mass": curr_member["mass"],
        "shape": curr_member["shape"],
        "scale": curr_member["scale"],
        "rotation": {
            "x": curr_rotate.x,
            "y": curr_rotate.y,
            "z": curr_rotate.z,
            "w": curr_rotate.w,
        },
        "translation": {
            "x": curr_translate.x,
            "y": curr_translate.y,
            "z": curr_translate.z,
        },
        "friction": friction,
        "ignore_collision": ignore_col,
    }

    new_members.append(new_member)

    # Constraint
    if curr_constraint is not None:

        if curr_constraint["constraint_type"] == "hinge":
            parent_frame = curr_constraint["frame_in_parent"]
            child_frame = curr_constraint["frame_in_child"]
        elif curr_constraint["constraint_type"] == "fixed":
            parent_frame = curr_constraint["attach_in_parent"]
            child_frame = curr_constraint["attach_in_child"]
        else:
            raise Exception("Unknown constraint type")

        p_c_tr, p_c_rot, _ = decompose(to_mat4(parent_frame))
        c_c_tr, c_c_rot, _ = decompose(to_mat4(child_frame))

        new_constraint = {
            "parent_name": "_".join(parent_names),
            "child_name": "_".join(new_name),
            "frame_in_parent": {
                "translation": {
                    "x": p_c_tr.x,
                    "y": p_c_tr.y,
                    "z": p_c_tr.z,
                },
                "rotation": {
                    "x": p_c_rot.x,
                    "y": p_c_rot.y,
                    "z": p_c_rot.z,
                    "w": p_c_rot.w,
                }
            },
            "frame_in_child": {
                "translation": {
                    "x": c_c_tr.x,
                    "y": c_c_tr.y,
                    "z": c_c_tr.z,
                },
                "rotation": {
                    "x": c_c_rot.x,
                    "y": c_c_rot.y,
                    "z": c_c_rot.z,
                    "w": c_c_rot.w,
                }
            },
            "type": curr_constraint["constraint_type"],
        }

        if curr_constraint["constraint_type"] == "hinge":
            new_constraint["limit_degree"] = curr_constraint["limit_degree"]

        new_constraints.append(new_constraint)

    for constraint in curr_member["children"]:
        queue.append((constraint["child_member"], curr_model_mat, new_name, constraint))

In [ ]:
new_robot = {
    "robot_name": "spider_new",
    "root_name": "body",
    "members": new_members,
    "constraints": new_constraints,
    "muscles": robot_data["muscles"],
}

In [ ]:
with open("./new_format_spider.json", "w") as f:
    f.write(json.dumps(new_robot, indent=4))